In [1]:
import sys
from torch.autograd import Variable
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import sklearn
import random
import time
import torch.utils.data

sys.path.append("../src/")
import data_reader as dr
from evaluate import Evaluation
from lstm import LSTM
from loss_function import loss_function
from loss_function import cs
from meter import AUCMeter

In [2]:
def normalize(x, dim):
    l2 = torch.norm(x, 2, dim).expand_as(x)
    return x / l2.clamp(min = 1e-8)

In [3]:
corpus_path = "../data_Android/corpus.tsv.gz"

corpus = dr.read_corpus(corpus_path)

embedding_path = "../data/glove.combined.300d.txt.gz"
embedding_tensor, word_to_indx = dr.getEmbeddingTensor(embedding_path)

ids_corpus = dr.map_corpus(corpus, word_to_indx, kernel_width = 1)

In [4]:
train_corpus_path = "../data/text_tokenized.txt.gz"
train_corpus = dr.read_corpus(train_corpus_path)
train_ids_corpus = dr.map_corpus(train_corpus, word_to_indx, kernel_width = 1)
train_path = "../data/train_random.txt"
train = dr.read_annotations(train_path)
train_ex = dr.create_train_set(train_ids_corpus, train)

In [5]:
import torch.utils.data

def train_model(train_data, dev_data, model):
    model.cuda()
    optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.0004, weight_decay = 1e-5)
    model.train()

    lasttime = time.time()
    for epoch in range(1, 31):
        print("-------------\nEpoch {}:\n".format(epoch))

        loss = run_epoch(train_data, True, model, optimizer, 40)
        #return loss
        print('Train loss: {:.6f}'.format(loss))
        torch.save(model, "model{}".format(epoch))
        
        AUC = run_epoch(dev_data, False, model, optimizer, 5)
        print('Val AUC: {:.6f}'.format(AUC))
        
        print('This epoch took: {:.6f}'.format(time.time() - lasttime))
        lasttime = time.time()

        
def run_epoch(data, is_training, model, optimizer, batch_size):
    '''
    Train model for one pass of train data, and return loss, acccuracy
    '''
    data_loader = torch.utils.data.DataLoader(
        data,
        batch_size=batch_size,
        shuffle=True,
        num_workers=4,
        drop_last=False)

    losses = []
    targets = []
    scores = []

    if is_training:
        model.train()
    else:
        model.eval()

    for batch in data_loader:
        pid_title = torch.unsqueeze(Variable(batch['pid_title']), 1)
        pid_body = torch.unsqueeze(Variable(batch['pid_body']), 1)
        rest_title = Variable(batch['rest_title'])
        rest_body = Variable(batch['rest_body'])
        
        pid_title_pad = torch.unsqueeze(Variable(batch['pid_title_pad']), 1)
        pid_body_pad = torch.unsqueeze(Variable(batch['pid_body_pad']), 1)
        rest_title_pad = Variable(batch['rest_title_pad'])
        rest_body_pad = Variable(batch['rest_body_pad'])
        
        pid_title, pid_body = pid_title.cuda(), pid_body.cuda()
        rest_title, rest_body = rest_title.cuda(), rest_body.cuda()
        pid_title_pad, pid_body_pad = pid_title_pad.cuda(), pid_body_pad.cuda()
        rest_title_pad, rest_body_pad = rest_title_pad.cuda(), rest_body_pad.cuda()
        
        if is_training:
            optimizer.zero_grad()
        
        pt = model(pid_title)
        pb = model(pid_body)
        rt = model(rest_title)
        rb = model(rest_body)
        
        pt = normalize(pt, 2)
        pb = normalize(pb, 2)
        rt = normalize(rt, 2)
        rb = normalize(rb, 2)
        
        # we need to take the mean pooling taking into account the padding
        # tensors are of dim batch_size x samples x output_size x (len - kernel + 1)
        # pad tensors are of dim batch_size x samples x (len - kernel + 1)
        
        pid_title_pad_ex = torch.unsqueeze(pid_title_pad, 2).expand_as(pt)
        pid_body_pad_ex = torch.unsqueeze(pid_body_pad, 2).expand_as(pb)
        rest_title_pad_ex = torch.unsqueeze(rest_title_pad, 2).expand_as(rt)
        rest_body_pad_ex = torch.unsqueeze(rest_body_pad, 2).expand_as(rb)
        
        pt = torch.squeeze(torch.sum(pt * pid_title_pad_ex, dim = 3), dim = 3)
        pb = torch.squeeze(torch.sum(pb * pid_body_pad_ex, dim = 3), dim = 3)
        rt = torch.squeeze(torch.sum(rt * rest_title_pad_ex, dim = 3), dim = 3)
        rb = torch.squeeze(torch.sum(rb * rest_body_pad_ex, dim = 3), dim = 3)

        # tensors are of dim batch_size x samples x output_size
        # need to scale down because not all uniformly padded
        
        ptp_norm = torch.sum(pid_title_pad, dim = 2).clamp(min = 1).expand_as(pt)
        pbp_norm = torch.sum(pid_body_pad, dim = 2).clamp(min = 1).expand_as(pb)
        rtp_norm = torch.sum(rest_title_pad, dim = 2).clamp(min = 1).expand_as(rt)
        rbp_norm = torch.sum(rest_body_pad, dim = 2).clamp(min = 1).expand_as(rb)
        
        pt = pt / ptp_norm
        pb = pb / pbp_norm
        rt = rt / rtp_norm
        rb = rb / rbp_norm
        
        pid_tensor = (pt + pb)/2
        rest_tensor = (rt + rb)/2
        
        if is_training:
            dropout = nn.Dropout(p = 0.2)
            # we don't need to re-scale these on eval because its just cos sim
            pid_tensor = dropout(pid_tensor)
            rest_tensor = dropout(rest_tensor)
        
        if is_training:
            loss = loss_function(pid_tensor, rest_tensor, margin = 1.0)
            loss.backward()
            losses.append(loss.cpu().data[0])
            optimizer.step()
        else:
            expanded = pid_tensor.expand_as(rest_tensor)
            similarity = cs(expanded, rest_tensor, dim=2).squeeze(2)
            similarity = torch.FloatTensor(similarity.data.cpu().numpy())
            labels = batch['labels']
            
            for sim in similarity:
                scores.append(sim)
            targets.extend(labels.view(-1))

    # Calculate epoch level scores
    if is_training:
        avg_loss = np.mean(losses)
        return avg_loss
    else:
        aucmeter = AUCMeter()
        aucmeter.reset()
        
        output = torch.cat(scores)
        expect = torch.LongTensor(targets)
        aucmeter.add(output, expect)
        return aucmeter.value(max_fpr=0.05)

In [6]:
#model = torch.load("cnn_model") # error when loading
model = LSTM(200, embedding_tensor, 0.2)
#model = torch.load("lstm_model")

In [7]:
eval_path = "dev.txt"
#eval_path = "../data_Android/dev_android.txt"
eval_anno = dr.read_annotations(eval_path, K_neg = -1, prune_pos_cnt = -1, ignore_dup = True)
eval_set = dr.create_dev_set(ids_corpus, eval_anno)

In [8]:
z = train_model(train_ex, eval_set, model)

-------------
Epoch 1:

Train loss: 0.970644
Val AUC: 0.580141
This epoch took: 106.888101
-------------
Epoch 2:

Train loss: 0.869042
Val AUC: 0.542372
This epoch took: 105.823376
-------------
Epoch 3:

Train loss: 0.821928
Val AUC: 0.532681
This epoch took: 105.281816
-------------
Epoch 4:

Train loss: 0.789277
Val AUC: 0.518764
This epoch took: 105.641343
-------------
Epoch 5:



Process Process-34:
Process Process-35:
Process Process-33:
Process Process-36:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/sunl/.conda/envs/my_pytorch/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/sunl/.conda/envs/my_pytorch/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/home/sunl/.conda/envs/my_pytorch/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/sunl/.conda/envs/my_pytorch/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/home/sunl/.conda/envs/my_pytorch/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/home/sunl/.conda/envs/my_pytorch/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 35, in _worker_loop
    r = i

KeyboardInterrupt: 

In [10]:
model = torch.load("model1")
z = run_epoch(eval_set, False, model, None, 5)
print(z)

0.580140985257


In [11]:
model = torch.load("model1")
eval_path = "test.txt"
eval_anno = dr.read_annotations(eval_path, K_neg = -1, prune_pos_cnt = -1, ignore_dup = True)
eval_set = dr.create_dev_set(ids_corpus, eval_anno)

z = run_epoch(eval_set, False, model, None, 5)
print(z)

0.565994666816


In [11]:
out = dr.build_android_qsets("../data_Android/test.pos.txt", "../data_Android/test.neg.txt")

In [12]:
qIDs, qCandidates, labels = out

In [13]:
f = open("test.txt", "w")

for qid in qIDs:
    l = len(labels[qid])
    if l == 101:
        qPos = [qCandidates[qid][0]]
        qNeg = qCandidates[qid][1:]
    else:
        qPos = qCandidates[qid][0:2]
        qNeg = qCandidates[qid][2:]
    for i in range(len(qPos)):
        line = qid + "\t" + qPos[i] + "\t" + " ".join(qNeg[100 * i: 100 * i + 100]) + "\n"
        f.write(line)
        
f.close()

In [21]:
lstm_parameters = list(model.parameters())
lstm_num_params = sum([np.prod(p.size()) for p in lstm_parameters]) - 126219 * 301 # embedding layer
print(lstm_num_params)

322400


In [30]:
list(ids_corpus.keys())[:10]

['1', '2', '5', '8', '9', '11', '16', '17', '27', '30']